<a href="https://colab.research.google.com/github/dhruvsuri8106-code/ECON-3916---Statistical-and-Machine-Learning/blob/main/Lab%206/Lab_6_Archietecture_of_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [14]:
df = sns.load_dataset('titanic')
df


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [11]:
print(f"Total Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

Total Population: 891
Population Survival Rate: 0.3838


In [7]:
np.random.seed(2026)
indices = np.random.permutation(df.index)

In [17]:
split_point = int(len(df) * 0.8)
train_idx = indices[:split_point]
test_idx = indices[split_point:]

train_set = df.loc[train_idx]
test_set = df.loc[test_idx]

train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train survival rate: {train_surv:.4f}")
print(f"Test survival rate: {test_surv:.4f}")
print(f"Delta: {delta:.4f}")
print


Train survival rate: 0.3736
Test survival rate: 0.4246
Delta: 0.0510


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [18]:
df.corr(numeric_only = True)

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,-0.557080,-0.203367
pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,0.094035,0.135207
age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,0.280328,0.198270
sibsp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,-0.253586,-0.584471
parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,-0.349943,-0.583398
fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,-0.182024,-0.271832
adult_male,-0.557080,0.094035,0.280328,-0.253586,-0.349943,-0.182024,1.000000,0.404744
alone,-0.203367,0.135207,0.198270,-0.584471,-0.583398,-0.271832,0.404744,1.000000


In [19]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test = train_test_split(df, test_size=0.2, stratify= df['pclass'], random_state= 42)

print("\n--- Stratified Split ---")
print("Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print("Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.550562
1    0.242978
2    0.206461
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.553073
1    0.240223
2    0.206704
Name: proportion, dtype: float64


In [26]:
import numpy as np
from scipy.stats import chisquare

# 1) Observed and expected counts
observed = np.array([450, 550])          # [Control, Treatment]
expected = np.array([500, 500])          # planned 50/50 split across 1000 users

# 2) Chi-square test
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

# Report
print(f"Observed: {observed.tolist()} | Expected: {expected.tolist()}")
print(f"Chi-square statistic: {chi2_stat:.6f}")
print(f"p-value: {p_value:.6g}")

# 3) Conclusion rule
if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.")
else:
    print("Variance is within natural limits.")


Observed: [450, 550] | Expected: [500, 500]
Chi-square statistic: 10.000000
p-value: 0.0015654
CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.


Why 550/450 isn’t just “bad luck”:



If assignment is truly random with a 50/50 split over 1000 users, you expect ~500/500 with typical random fluctuation of about ±16 users (that’s the binomial standard deviation, √(1000·0.5·0.5) ≈ 15.8).


You observed +50 / -50, which is about 50 / 15.8 ≈ 3.16 standard deviations away from expectation—rare under honest randomization.


The chi-square test quantifies that rarity: it gives the probability of seeing a deviation at least this large if the split were truly 50/50. A very small p-value (especially < 0.01) strongly suggests a systematic issue (routing, eligibility filters, bucketing bug, logging loss), not mere chance.

